# Explorando tendencias de mercado de Airbnb

La ciudad de Nueva York tiene una variedad de anuncios de Airbnb para satisfacer la alta demanda de alojamiento temporal para viajeros, con diferentes niveles de precios, tipos de habitaciones y ubicaciones.

La ciudad de Nueva York, es una de las ciudades más visitadas del mundo. Hay muchos anuncios de Airbnb en la ciudad de Nueva York para satisfacer la alta demanda de alojamiento temporal para viajeros, que puede durar desde unas pocas noches hasta muchos meses. En este proyecto, analizaremos más de cerca el mercado de Airbnb de Nueva York combinando datos de varios tipos de archivos como .csv, .tsv y .xlsx.

Recuerde que los archivos CSV, TSV y Excel son tres formatos comunes para almacenar datos. Tiene a su disposición [tres archivos](https://tajamar365.sharepoint.com/:f:/s/3405-MasterIA2024-2025/EoXMgjAJLLNJqWDyKDig8kABSUkpWj_HGQjl267qNnHS9g?e=foxjFc) que contienen datos sobre los anuncios de Airbnb de 2019:  

- `airbnb_price.csv `: Este es un archivo CSV que contiene datos sobre precios y ubicaciones de anuncios de Airbnb.  
  1. `listing_id`: identificador único del anuncio  
  2. `price`: precio del anuncio por noche en USD  
  3. `nbhood_full`: nombre del distrito y del barrio donde se encuentra el anuncio  
    

- `airbnb_room_type.xlsx`: Este es un archivo Excel que contiene datos sobre las descripciones de los anuncios de Airbnb y los tipos de habitaciones.  
  1. `listing_id`: identificador único del anuncio  
  2. `description`: descripción del anuncio     
  3. `room_type`: Airbnb tiene tres tipos de habitaciones: habitaciones compartidas, habitaciones privadas y casas o apartamentos completos 


- `airbnb_last_review.tsv`: Este es un archivo TSV que contiene datos sobre los nombres de los anfitriones de Airbnb y las fechas de revisión.  
  1. `listing_id`: identificador único del anuncio  
  2. `host_name`:  nombre del anfitrión del anuncio  
  3. `last_review`: echa en la que se revisó el anuncio por última vez   

Como consultor que trabaja para una empresa inmobiliaria emergente, ha recopilado datos de anuncios de Airbnb de varias fuentes para investigar el mercado de alquileres a corto plazo en Nueva York. Analizará estos datos para brindar información sobre habitaciones privadas a la empresa inmobiliaria.

## NOTA: El fichero excel no funciona, se debe convertir a .csv

In [0]:
from pyspark.sql import SparkSession

# Inicia la sesión de Spark
spark = SparkSession.builder.appName("AirbnbData").getOrCreate()

# Leer archivos .tsv y .csv
df_lastreview = spark.read.option("delimiter", "\t").csv("dbfs:/FileStore/shared_uploads/Examen_practico_3/Caso_3/airbnb_last_review.tsv", header=True, inferSchema=True)
df_price = spark.read.option("delimiter", ",").csv("dbfs:/FileStore/shared_uploads/Examen_practico_3/Caso_3/airbnb_price.csv", header=True, inferSchema=True)

# Leer archivo .xlsx (se necesita instalar la librería spark-excel)
df_roomtype = spark.read.option("delimiter", ";").csv("dbfs:/FileStore/shared_uploads/Examen_practico_3/Caso_3/airbnb_room_type.csv", header=True, inferSchema=True)

# Mostrar las primeras filas de cada DataFrame
df_lastreview.show(5)
df_price.show(5)
df_roomtype.show(5)

+----------+-----------+------------+
|listing_id|  host_name| last_review|
+----------+-----------+------------+
|      2595|   Jennifer| May 21 2019|
|      3831|LisaRoxanne|July 05 2019|
|      5099|      Chris|June 22 2019|
|      5178|   Shunichi|June 24 2019|
|      5238|        Ben|June 09 2019|
+----------+-----------+------------+
only showing top 5 rows

+----------+-----------+--------------------+
|listing_id|      price|         nbhood_full|
+----------+-----------+--------------------+
|      2595|225 dollars|  Manhattan, Midtown|
|      3831| 89 dollars|Brooklyn, Clinton...|
|      5099|200 dollars|Manhattan, Murray...|
|      5178| 79 dollars|Manhattan, Hell's...|
|      5238|150 dollars|Manhattan, Chinatown|
+----------+-----------+--------------------+
only showing top 5 rows

+----------+--------------------+---------------+
|listing_id|         description|      room_type|
+----------+--------------------+---------------+
|      2595|Skylit Midtown Ca...|Entire home

In [0]:

#COnvertir los dataframe a pandas para mayor comodidad
df_lastreview = df_lastreview.toPandas()
df_price = df_price.toPandas()
df_roomtype = df_roomtype.toPandas()

In [0]:
# Convertir 'listing_id' a tipo string en todos los DataFrames
df_lastreview['listing_id'] = df_lastreview['listing_id'].astype(str)
df_price['listing_id'] = df_price['listing_id'].astype(str)
df_roomtype['listing_id'] = df_roomtype['listing_id'].astype(str)

# Realizar el merge
df_airbnb = df_lastreview.merge(df_price, on='listing_id').merge(df_roomtype, on='listing_id')


In [0]:
df_airbnb = df_lastreview.merge(df_price, on='listing_id').merge(df_roomtype,on='listing_id')
df_airbnb.display()

listing_id,host_name,last_review,price,nbhood_full,description,room_type
2595,Jennifer,May 21 2019,225 dollars,"Manhattan, Midtown",Skylit Midtown Castle,Entire home/apt
3831,LisaRoxanne,July 05 2019,89 dollars,"Brooklyn, Clinton Hill",Cozy Entire Floor of Brownstone,Entire home/apt
5099,Chris,June 22 2019,200 dollars,"Manhattan, Murray Hill",Large Cozy 1 BR Apartment In Midtown East,Entire home/apt
5178,Shunichi,June 24 2019,79 dollars,"Manhattan, Hell's Kitchen",Large Furnished Room Near B'way,private room
5238,Ben,June 09 2019,150 dollars,"Manhattan, Chinatown",Cute & Cozy Lower East Side 1 bdrm,Entire home/apt
5295,Lena,June 22 2019,135 dollars,"Manhattan, Upper West Side",Beautiful 1br on Upper West Side,Entire home/apt
5441,Kate,June 23 2019,85 dollars,"Manhattan, Hell's Kitchen",Central Manhattan/near Broadway,Private room
5803,Laurie,June 24 2019,89 dollars,"Brooklyn, South Slope","Lovely Room 1, Garden, Best Area, Legal rental",Private room
6021,Claudio,July 05 2019,85 dollars,"Manhattan, Upper West Side",Wonderful Guest Bedroom in Manhattan for SINGLES,Private room
6848,Allen & Irina,June 29 2019,140 dollars,"Brooklyn, Williamsburg",Only 2 stops to Manhattan studio,entire home/apt


In [0]:
#Limpiar el valor del precio para poder calcular la media posteriormente
df_airbnb["price"] = df_airbnb["price"].str.replace(" dollars", "").astype(int)

- ¿Cuáles son las fechas de las primeras y las últimas reseñas? Almacene estos valores como dos variables independientes con los nombres que prefiera.
- ¿Cuántos de los anuncios son habitaciones privadas? Guárdelo en cualquier variable.
- ¿Cuál es el precio promedio de los anuncios? Redondee a los dos decimales más cercanos y guárdelo en una variable.
- Combine las nuevas variables en un DataFrame llamado review_dates con cuatro columnas en el siguiente orden: first_reviewed, last_reviewed, nb_private_rooms y avg_price. El DataFrame solo debe contener una fila de valores.

In [0]:
#Obtener las primeras y ultimas fechas con min() y max()
first_reviewed = df_airbnb["last_review"].min()
last_reviewed = df_airbnb["last_review"].max()

print("Más nueva: " + str(last_reviewed) + ", más vieja: " + str(first_reviewed))


Más nueva: April 01 2019, más vieja: May 31 2019


In [0]:
#Extraer el número de habitaciones privadas
nb_private_rooms = (df_airbnb["room_type"].str.lower() == "private room").sum()
print("Total de habitaciones privadas: " + str(nb_private_rooms))

Total de habitaciones privadas: 11282


In [0]:
#Cálculo de la media de precio de las habitaciones
avg_price = (df_airbnb["price"]).mean()
print("Precio medio de las habitaciones: " + '{0:.2f}'.format(avg_price))

Precio medio de las habitaciones: 141.78


In [0]:
#Crear un dataframe con los valores obtenidos
df_airbnb_final = pd.DataFrame(
    {
        "first_reviewed": [first_reviewed],
        "last_reviewed": [last_reviewed],
        "nb_private_rooms": [nb_private_rooms],
        "avg_price": [round(avg_price, 2)]
    }
)

df_airbnb_final.display()


first_reviewed,last_reviewed,nb_private_rooms,avg_price
April 01 2019,May 31 2019,11282,141.78
